In [1]:
# inputs

# set to True when using future work
# v_next = False should represent what core can do today
v_next = True

# set to True when modeling validators (uses archives)
# watchers expose endpoints for RPC that need to be modeled
is_validator = False

# set to True when comnputing limits
# this uses settings that are "worst case"
model_max = True

# ledger cycle period in seconds
ledger_time = 5

# Transactions Per Ledger (derived from TPS as it's the common metric)
if model_max:
    classic_TPL = 5000*ledger_time
    soroban_TPL = 2000*ledger_time
else:
    classic_TPL = 1000*ledger_time
    soroban_TPL = 20*ledger_time

# how long each tx type takes
classic_tx_exec_time_ms = 0.1
soroban_tx_exec_time_ms = 10

if v_next:
    # number of threads used during apply
    nb_exec_lanes = 50
    apply_time_ms = 3000
else:
    apply_time_ms = 1000

# how long is allocated for writing
# (may want to make this a function of classic/Soroban tps&footprints)
if model_max:
    apply_write_time_ms = 250
else:
    apply_write_time_ms = 100

# tx flooding multiplier (how many unique transactions get received in steady state)
flooding_recv_factor = 2.5

# other settings
# apply time (in addition to source account modified for seqnum & fees)

if model_max:
    #   classic
    #      max number of LE reads per op (vnext: limited DEX)
    classic_LE_reads_per_op = 10 if v_next else 1000
    #      max number of LE writes per op
    classic_LE_writes_per_op = classic_LE_reads_per_op
    #      avg size of LE
    classic_LE_size = 140

    #   soroban
    #      max number of LE reads
    soroban_LE_reads_per_op = 40
    #      max number of LE writes
    soroban_LE_writes_per_op = 25
    # write as much as possible, use max allowed per tx
    soroban_LE_size = (65*1024)/soroban_LE_writes_per_op
    # tps getledgerentries (watcher node)
    gle_tps = 5
else:
    #   classic
    #      avg number of LE reads per op 
    classic_LE_reads_per_op = 5
    #      avg number of LE writes per op
    classic_LE_writes_per_op = 5
    #      avg size of LE
    classic_LE_size = 140

    #   soroban
    #      avg number of LE reads
    soroban_LE_reads_per_op = 10
    #      avg number of LE writes
    soroban_LE_writes_per_op = 5
    # use a 10th of capacity
    soroban_LE_size = (65*1024)/soroban_LE_writes_per_op/10
    # tps getledgerentries (watcher node)
    gle_tps = 2


In [2]:
# bucket list settings

# number of iops and bytes needed to read a single ledger entry
# need to
#    1. walk from recent to oldest buckets (up to bucket 19)
#        bloom filter helps short circuit lookups
#    2. for each bucket, page size on larger buckets -> more than 1 read occurs

# additional extra reads (overhead)
bl_avg_extra_nb_reads = 1

if model_max:
    # more Soroban entries in max mode
    bl_avg_le_size = 1024
else:
    bl_avg_le_size = 300


bl_avg_extra_bytes_read = bl_avg_le_size*bl_avg_extra_nb_reads

bl_avg_nb_reads = 1+bl_avg_extra_nb_reads

def bl_bytes_read_per_le(le_size):
    return le_size + bl_avg_extra_bytes_read

In [3]:

# calculate activity per ledger (apply time)

# size of an account LE
le_account_bytes = 127

# fees/seqnum
ledger_LE_reads = classic_TPL*bl_avg_nb_reads
ledger_bytes_read = classic_TPL*bl_bytes_read_per_le(le_account_bytes)
ledger_LE_writes = classic_TPL
ledger_bytes_write = ledger_LE_writes * le_account_bytes

print("ledger_LE_reads: ", ledger_LE_reads)
print("ledger_bytes_read: ", ledger_bytes_read)

print("ledger_LE_writes: ", ledger_LE_writes)
print("ledger_bytes_write: ", ledger_bytes_write)

# per phase
ledger_classic_LE_reads = classic_TPL*classic_LE_reads_per_op
ledger_classic_bytes_read = ledger_classic_LE_reads*bl_bytes_read_per_le(classic_LE_size)
ledger_classic_LE_reads *= bl_avg_nb_reads

print("ledger_classic_LE_reads: ", ledger_classic_LE_reads)
print("ledger_classic_bytes_read: ", ledger_classic_bytes_read)

ledger_classic_LE_writes = classic_TPL*classic_LE_writes_per_op
ledger_classic_bytes_write = ledger_classic_LE_writes*classic_LE_size

print("ledger_classic_LE_writes: ", ledger_classic_LE_writes)
print("ledger_classic_bytes_write: ", ledger_classic_bytes_write)

ledger_soroban_LE_reads = soroban_TPL*soroban_LE_reads_per_op
ledger_soroban_bytes_read = ledger_soroban_LE_reads*bl_bytes_read_per_le(soroban_LE_size)
ledger_soroban_LE_reads *= bl_avg_nb_reads

print("ledger_soroban_LE_reads: ", ledger_soroban_LE_reads)
print("ledger_soroban_bytes_read: ", ledger_soroban_bytes_read)


ledger_soroban_LE_writes = soroban_TPL*soroban_LE_writes_per_op
ledger_soroban_bytes_write = ledger_soroban_LE_writes*soroban_LE_size

print("ledger_soroban_LE_writes: ", ledger_soroban_LE_writes)
print("ledger_soroban_bytes_write: ", ledger_soroban_bytes_write)


ledger_LE_reads += ledger_classic_LE_reads + ledger_soroban_LE_reads
ledger_bytes_read += ledger_classic_bytes_read + ledger_soroban_bytes_read
ledger_LE_writes += ledger_classic_LE_writes + ledger_soroban_LE_writes
ledger_bytes_write += ledger_classic_bytes_write + ledger_soroban_bytes_write


ledger_LE_reads:  50000
ledger_bytes_read:  28775000
ledger_LE_writes:  25000
ledger_bytes_write:  3175000
ledger_classic_LE_reads:  500000
ledger_classic_bytes_read:  291000000
ledger_classic_LE_writes:  250000
ledger_classic_bytes_write:  35000000
ledger_soroban_LE_reads:  800000
ledger_soroban_bytes_read:  1474560000.0
ledger_soroban_LE_writes:  250000
ledger_soroban_bytes_write:  665600000.0


In [4]:
# Overlay activity per ledger period

##   bucket list
ledger_overlay_LE_reads = ((classic_TPL+soroban_TPL)*flooding_recv_factor)
ledger_overlay_LE_bytes = ledger_overlay_LE_reads*bl_bytes_read_per_le(le_account_bytes)
ledger_overlay_LE_reads *= bl_avg_nb_reads

##   bandwdith

if v_next:
    overlay_fan_out = 10
else:
    overlay_fan_out = 20

if model_max:
    tier1_size = 100
else:
    tier1_size = 25

# how long is allocated per ledger to flooding
ledger_overlay_processing_time_ms = ledger_time*1000

classic_tx_bytes = 200
soroban_tx_footprint = soroban_LE_reads_per_op+soroban_LE_writes_per_op

# Soroban key size (as seen in footprints)
soroban_LE_key_size_bytes = 80
# Soroban tx size without footprint
soroban_tx_no_footprint_size_bytes = 200

soroban_tx_bytes = soroban_tx_no_footprint_size_bytes + (soroban_tx_footprint*soroban_LE_key_size_bytes)

scp_message_bytes = 150
# number of tx sets per ledger (should be 1, but due to timing issues more than 1 leader may nominate)
ledger_scp_tx_set_count = 1.1

### tx flooding
##### use the same factor than recv
ledger_classic_tx_flooding = (classic_TPL*flooding_recv_factor)
ledger_soroban_tx_flooding = (soroban_TPL*flooding_recv_factor)
ledger_tx_flood_bytes = (ledger_classic_tx_flooding*classic_tx_bytes) + (ledger_soroban_tx_flooding*soroban_tx_bytes)
print("flooding bytes (per connection): ", ledger_tx_flood_bytes)
ledger_tx_flood_bytes *= overlay_fan_out

### SCP flooding
#### 6 messages per tier1 org
ledger_scp_messages = tier1_size*6
ledger_scp_flood_bytes = ledger_scp_messages*scp_message_bytes
#### tx set overhead per ledger
ledger_txset_bytes = (classic_TPL*classic_tx_bytes) + (soroban_TPL*soroban_tx_bytes)
print("TxSet bytes: ", ledger_txset_bytes)
ledger_txset_flood_bytes = ledger_txset_bytes*ledger_scp_tx_set_count
ledger_scp_flood_bytes += ledger_txset_flood_bytes

ledger_scp_flood_bytes *= overlay_fan_out

# totals
ledger_flood_bytes = ledger_tx_flood_bytes+ledger_scp_flood_bytes

flood_bps = ledger_flood_bytes / ledger_overlay_processing_time_ms * 1000

print("Network bps (GBit/s): ", flood_bps*8/1024/1024/1024)


flooding bytes (per connection):  147500000.0
TxSet bytes:  59000000
Network bps (GBit/s):  3.1663477420806885


In [5]:
# calculates additional "overlay activity" caused by watcher node activity
if not is_validator:
    # watchers expose the "getledgerentries" endpoint
    gle_reads = ledger_time*2
    gle_read_bytes = gle_reads*bl_bytes_read_per_le(bl_avg_le_size)
    gle_reads *= bl_avg_nb_reads

    ledger_overlay_LE_reads += gle_reads
    ledger_overlay_LE_bytes += gle_read_bytes

In [6]:
# apply is equivalent to the sequence [loads, classic, Soroban, writes]

if v_next:
    # overlay can use the entire time (background apply)
    ledger_overlay_time_ms = ledger_time*1000
    # calculates time available to read by assuming parallel execution
    apply_read_time_ms = apply_time_ms - apply_write_time_ms - (classic_TPL*classic_tx_exec_time_ms+soroban_TPL*soroban_tx_exec_time_ms)/nb_exec_lanes
    assert apply_read_time_ms > 0, "TPS too high (apply_read_time_ms <= 0)"
else:
    ledger_overlay_time_ms = ledger_time*1000 - apply_time_ms
    apply_read_time_ms = apply_time_ms - apply_write_time_ms - (classic_TPL*classic_tx_exec_time_ms+soroban_TPL*soroban_tx_exec_time_ms)
    assert apply_read_time_ms > 0, "TPS too high (apply_read_time_ms <= 0)"

# calculates max iops for overlay and apply separately

iops_overlay = ledger_overlay_LE_reads*1000/ledger_overlay_time_ms
iops_apply = ledger_LE_reads*1000/apply_read_time_ms

rbps_overlay = ledger_overlay_LE_bytes*1000/ledger_overlay_time_ms

rbps_apply = ledger_bytes_read*1000/apply_read_time_ms
wbps_apply = ledger_bytes_write*1000/apply_write_time_ms



if v_next:
    # parallel -> add
    max_iops = iops_overlay + iops_apply
    max_rbps = rbps_overlay + rbps_apply
    max_wbps = wbps_apply
    max_bps = max(rbps_overlay + rbps_apply, rbps_overlay + wbps_apply)
else:
    # sequential -> max
    max_iops = max(iops_overlay, iops_apply)
    max_rbps = max(rbps_overlay, rbps_apply)
    max_wbps = wbps_apply
    max_bps = max(max_rbps, max_wbps)

print("max iops: ", max_iops)
print("max read Mbps: ", max_rbps/1024/1024)
print("max write Mbps: ", max_wbps/1024/1024)
print("max disk bandwidth Mbps: ", max_bps/1024/1024)

# source: https://aws.amazon.com/ec2/instance-types/i3en
max_4k_IOPS_NVMe = 2000000
max_write_NVMe = 16*1024*1024*1024
assert max_iops < max_4k_IOPS_NVMe, "disk IOPS exceeded"
assert max_bps < (4096*max_4k_IOPS_NVMe), "disk bandwidth exceeded"
assert max_wbps < max_write_NVMe, "disk linear write bandwidth exceeded"


max iops:  1963575.4285714286
max read Mbps:  2463.800726209368
max write Mbps:  2684.6885681152344
max disk bandwidth Mbps:  2703.9018592834473
